In [3]:
!pip3 install numpy scikit-learn transformers torch scipy

Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 575.0 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 25.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 30.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 28.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 2.6 MB/s eta 0:00:0000:0100:09m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:0000:0100:04m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage, fcluster
from transformers import AutoTokenizer, AutoModel
import torch

/home/vscode/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
import re
def _split_sentences(text):
    sentences = re.split(r'(?<=[.?!])\s+', text)
    return sentences
  
def _combine_sentences(sentences):
    combined_sentences = []
    for i in range(len(sentences)):
        combined_sentence = sentences[i]
        if i > 0:
            combined_sentence = sentences[i-1] + ' ' + combined_sentence
        if i < len(sentences) - 1:
            combined_sentence += ' ' + sentences[i+1]
        combined_sentences.append(combined_sentence)
    return combined_sentences


def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
  
def get_embeddings(texts, model_name="VoVanPhuc/sup-SimCSE-VietNamese-phobert-base"):
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModel.from_pretrained(model_name)

  encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

  with torch.no_grad():
      model_output = model(**encoded_input)
  
  embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

  return embeddings.numpy()

In [11]:

def chunk_text(text, num_clusters=4, distance_threshold=None):

    single_sentences_list = _split_sentences(text)
    print(single_sentences_list)

    combined_sentences = _combine_sentences(single_sentences_list)
    print(combined_sentences)

    
    embeddings = get_embeddings(combined_sentences)


    distance_matrix = cosine_distances(embeddings)

    
    Z = linkage(distance_matrix, method='average')  # 'average' is for average linkage; you can also try 'ward', 'complete', etc.


    if num_clusters:
        cluster_labels = fcluster(Z, t=num_clusters, criterion='maxclust')
    elif distance_threshold:
        cluster_labels = fcluster(Z, t=distance_threshold, criterion='distance')
    else:
        raise ValueError("Either num_clusters or distance_threshold must be specified.")

    
    chunks = []
    current_chunk = []
    current_label = cluster_labels[0]

    for i, sentence in enumerate(single_sentences_list):
        if cluster_labels[i] == current_label:
            current_chunk.append(sentence)
        else:
            # Start a new chunk when the cluster label changes
            chunks.append(' '.join(current_chunk))
            current_chunk = [sentence]
            current_label = cluster_labels[i]

    # Append the last chunk
    if current_chunk:
        chunks.append(' '.join(current_chunk))

 
    return chunks

In [12]:


text = """
Quỹ Hỗ trợ đầu tư được tổ chức theo mô hình mới, tương tự một đơn vị sự nghiệp công lập nhưng có khác biệt. Cụ thể, quỹ này sẽ không cung cấp dịch vụ công, không có bộ máy riêng để quản lý và thực hiện các nhiệm vụ đặc thù trong chi hỗ trợ đầu tư. Hội đồng quản lý quỹ do Thủ tướng quyết định thành lập, nhiệm kỳ 5 năm và được xem xét bổ nhiệm lại. Cơ cấu, tổ chức của Hội đồng quản lý Quỹ thực hiện theo quyết định của Thủ tướng.
Liên quan tới chính sách hỗ trợ, theo Thứ trưởng Bích Ngọc, Quỹ Hỗ trợ đầu tư sẽ dành nguồn lực cho doanh nghiệp công nghệ cao (bán dẫn, AI...), đơn vị có dự án đầu tư sản xuất sản phẩm hoặc ứng dụng công nghệ cao; dự án đầu tư trung tâm nghiên cứu và phát triển (R&D).
Để được hỗ trợ, doanh nghiệp cần có dự án đầu tư sản xuất sản phẩm, ứng dụng công nghệ cao, với vốn tối thiểu 12.000 tỷ đồng hoặc doanh thu ít nhất 20.000 tỷ đồng một năm.
Doanh nghiệp có dự án trong lĩnh vực công nghiệp chip, mạch tích hợp bán dẫn, trung tâm dữ liệu AI cần có vốn tối thiểu 6.000 tỷ đồng hoặc doanh thu 10.000 tỷ đồng một năm.
Tiêu chí về vốn, doanh thu tối thiểu mỗi năm sẽ không áp dụng với doanh nghiệp có dự án đầu tư sản xuất sản phẩm, ứng dụng công nghệ cao thuộc danh mục công nghệ, sản phẩm đột phá được ưu tiên nghiên cứu, phát triển và đơn vị có dự án thiết kế vi mạch.
Khoản hỗ trợ từ quỹ này sẽ được chi trực tiếp bằng tiền và qua các hạng mục hỗ trợ:
"""
chunks = chunk_text(text)
for chunk in chunks:
    print(chunk,"\n----------------------------------------------------------------------------\n")
print(f"\n{len(chunks)} chunks")

['\nQuỹ Hỗ trợ đầu tư được tổ chức theo mô hình mới, tương tự một đơn vị sự nghiệp công lập nhưng có khác biệt.', 'Cụ thể, quỹ này sẽ không cung cấp dịch vụ công, không có bộ máy riêng để quản lý và thực hiện các nhiệm vụ đặc thù trong chi hỗ trợ đầu tư.', 'Hội đồng quản lý quỹ do Thủ tướng quyết định thành lập, nhiệm kỳ 5 năm và được xem xét bổ nhiệm lại.', 'Cơ cấu, tổ chức của Hội đồng quản lý Quỹ thực hiện theo quyết định của Thủ tướng.', 'Liên quan tới chính sách hỗ trợ, theo Thứ trưởng Bích Ngọc, Quỹ Hỗ trợ đầu tư sẽ dành nguồn lực cho doanh nghiệp công nghệ cao (bán dẫn, AI...), đơn vị có dự án đầu tư sản xuất sản phẩm hoặc ứng dụng công nghệ cao; dự án đầu tư trung tâm nghiên cứu và phát triển (R&D).', 'Để được hỗ trợ, doanh nghiệp cần có dự án đầu tư sản xuất sản phẩm, ứng dụng công nghệ cao, với vốn tối thiểu 12.000 tỷ đồng hoặc doanh thu ít nhất 20.000 tỷ đồng một năm.', 'Doanh nghiệp có dự án trong lĩnh vực công nghiệp chip, mạch tích hợp bán dẫn, trung tâm dữ liệu AI cần có

/tmp/ipykernel_1067/268849396.py:16: ClusterWarning: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  Z = linkage(distance_matrix, method='average')  # 'average' is for average linkage; you can also try 'ward', 'complete', etc.


In [13]:
from sklearn.metrics import silhouette_score
def determine_optimal_clusters(embeddings, max_clusters=10):
    distance_matrix = cosine_distances(embeddings)
   
    Z = linkage(distance_matrix, method='average')
   
    wcss = []
    for n_clusters in range(2, max_clusters + 1):
        cluster_labels = fcluster(Z, t=n_clusters, criterion='maxclust')
        wcss.append(calculate_wcss(embeddings, cluster_labels))
   
    total_variance = np.sum((embeddings - np.mean(embeddings, axis=0))**2)
    explained_variance = [1 - (w / total_variance) for w in wcss]
   
    optimal_clusters = find_elbow_point(range(2, max_clusters + 1), explained_variance)
   
    return optimal_clusters

def calculate_wcss(data, labels):
    n_clusters = len(set(labels))
    wcss = 0
    for i in range(n_clusters):
        cluster_points = data[labels == i+1]
        cluster_mean = np.mean(cluster_points, axis=0)
        wcss += np.sum((cluster_points - cluster_mean)**2)
    return wcss

def find_elbow_point(x, y):
    diffs = np.diff(y, 2)
    return x[np.argmax(diffs) + 1]

def chunk_text_with_clusters(text, num_clusters):
    single_sentences_list = _split_sentences(text)
    combined_sentences = _combine_sentences(single_sentences_list)
    embeddings = get_embeddings(combined_sentences)

    distance_matrix = cosine_distances(embeddings)
    Z = linkage(distance_matrix, method='average')
    cluster_labels = fcluster(Z, t=num_clusters, criterion='maxclust')

    chunks = []
    current_chunk = []
    current_label = cluster_labels[0]

    for i, sentence in enumerate(single_sentences_list):
        if cluster_labels[i] == current_label:
            current_chunk.append(sentence)
        else:
            chunks.append(' '.join(current_chunk))
            current_chunk = [sentence]
            current_label = cluster_labels[i]

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks


def chunk_text(text, max_clusters=10):
    single_sentences_list = _split_sentences(text)
    combined_sentences = _combine_sentences(single_sentences_list)
    embeddings = get_embeddings(combined_sentences)
   
    optimal_clusters = determine_optimal_clusters(embeddings, max_clusters)
   
    return chunk_text_with_clusters(text, num_clusters=optimal_clusters)

res = chunk_text(text)
res

/tmp/ipykernel_1067/912938840.py:5: ClusterWarning: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  Z = linkage(distance_matrix, method='average')
/tmp/ipykernel_1067/912938840.py:38: ClusterWarning: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  Z = linkage(distance_matrix, method='average')


['\nQuỹ Hỗ trợ đầu tư được tổ chức theo mô hình mới, tương tự một đơn vị sự nghiệp công lập nhưng có khác biệt. Cụ thể, quỹ này sẽ không cung cấp dịch vụ công, không có bộ máy riêng để quản lý và thực hiện các nhiệm vụ đặc thù trong chi hỗ trợ đầu tư. Hội đồng quản lý quỹ do Thủ tướng quyết định thành lập, nhiệm kỳ 5 năm và được xem xét bổ nhiệm lại.',
 'Cơ cấu, tổ chức của Hội đồng quản lý Quỹ thực hiện theo quyết định của Thủ tướng.',
 'Liên quan tới chính sách hỗ trợ, theo Thứ trưởng Bích Ngọc, Quỹ Hỗ trợ đầu tư sẽ dành nguồn lực cho doanh nghiệp công nghệ cao (bán dẫn, AI...), đơn vị có dự án đầu tư sản xuất sản phẩm hoặc ứng dụng công nghệ cao; dự án đầu tư trung tâm nghiên cứu và phát triển (R&D). Để được hỗ trợ, doanh nghiệp cần có dự án đầu tư sản xuất sản phẩm, ứng dụng công nghệ cao, với vốn tối thiểu 12.000 tỷ đồng hoặc doanh thu ít nhất 20.000 tỷ đồng một năm.',
 'Doanh nghiệp có dự án trong lĩnh vực công nghiệp chip, mạch tích hợp bán dẫn, trung tâm dữ liệu AI cần có vốn t